In [75]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import os


In [76]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gitadityamaddali/flipkart-laptop-reviews")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'flipkart-laptop-reviews' dataset.
Path to dataset files: /kaggle/input/flipkart-laptop-reviews


In [77]:
# ----- Step 1: Load data -----
# Example path: change this to your actual Kaggle dataset path
data_path = os.path.join(path, "laptops_dataset_final_600.csv")

# Read CSV file
data = pd.read_csv(data_path, encoding='utf-8')

In [78]:
# ----- Step 2: Select relevant columns -----
data = data[['product_name', 'overall_rating']].dropna()

In [79]:
# ----- Step 3: Create labels -----
# Good (1) if rating >= 4, else Bad (0)
data['label'] = data['overall_rating'].apply(lambda x: 1 if x >= 4 else 0)

In [80]:
texts = data['product_name'].astype(str).values
labels = data['label'].values

In [81]:
# Tokenize text (convert words to integers)
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [82]:
# Pad sequences to equal length
max_len = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_len, padding='post')


In [83]:
# ----- Step 5: Split data -----
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)


In [84]:
# ----- Step 6: Build RNN model -----
vocab_size = len(tokenizer.word_index) + 1

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=32, input_length=max_len),
    SimpleRNN(64, activation='tanh'),
    Dense(1, activation='sigmoid')  # 1 neuron for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [85]:
# ----- Step 7: Train the model -----
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
483/483 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9638 - loss: 0.1057 - val_accuracy: 0.9889 - val_loss: 0.0307
Epoch 2/10
483/483 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9974 - loss: 0.0109 - val_accuracy: 0.9961 - val_loss: 0.0158
Epoch 3/10
483/483 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9987 - loss: 0.0057 - val_accuracy: 0.9979 - val_loss: 0.0124
Epoch 4/10
483/483 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9995 - loss: 0.0024 - val_accuracy: 0.9984 - val_loss: 0.0118
Epoch 5/10
483/483 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9997 - loss: 0.0023 - val_accuracy: 0.9984 - val_loss: 0.0091
Epoch 6/10
483/483 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9997 - loss: 0.0014 - val_accuracy: 0.9987 - val_loss: 0.0076
Epoch 7/10
483/483 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 1.0000 - loss: 2.9067e-04 - val_accuracy: 0.9987 - val_loss: 0.0088
Epoch 8/10
483/483 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 1.0000 - loss: 5.7576e-05 - val_ac

In [86]:
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)        │ (None, 16, 32)         │        11,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_10 (SimpleRNN)       │ (None, 64)             │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,613 (205.52 KB)

 Trainable params: 17,537 (68.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 35,076 (137.02 KB)

In [87]:
# ----- Step 8: Evaluate on test data -----
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc*100:.2f}%")

151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9992 - loss: 0.0088
Test Accuracy: 99.92%


In [96]:
# ----- Step 9: Predict a new review -----
test_review = ["Lenovo Chromebook MediaTek Kompanio 520 - (4 GB/128 GB EMMC St..."]
seq = tokenizer.texts_to_sequences(test_review)
pad = pad_sequences(seq, maxlen=max_len, padding='post')
pred = model.predict(pad)[0][0]

if pred > 0.5:
    print("Predicted Sentiment: GOOD Review")
else:
    print("Predicted Sentiment: BAD Review")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Predicted Sentiment: BAD Review
